## Please Download the gguf file (model_path)

download this: Llama-3.2-1B-Instruct-IQ3_M.gguf
link: https://huggingface.co/bartowski/Llama-3.2-1B-Instruct-GGUF/blob/main/Llama-3.2-1B-Instruct-IQ3_M.gguf

Install some dependencies

In [ ]:
!pip install -q -U accelerate==0.27.1
!pip install -q -U datasets==2.17.0
!pip install -q -U transformers==4.38.1
!pip install langchain sentence-transformers chromadb langchainhub

!pip install langchain-community langchain-core

Get the Model You Want

In [ ]:
#!pip install --upgrade --upgrade-strategy eager "optimum[neural-compressor]"
!pip install llama-cpp-python

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb

import time
import uuid
from llama_cpp import Llama
#from fuzzywuzzy import fuzz
#from optimum.quantization import QuantizeruenceClassification


/opt/anaconda3/envs/CampusConnect/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Define Variables

In [2]:
from llama_cpp import Llama

model_path = "./Llama-3.2-1B-Instruct-IQ3_M.gguf"

model = Llama(model_path=model_path, n_ctx=2048, n_threads=8)


llama_model_load_from_file_impl: using device Metal (Apple M3 Pro) - 27647 MiB free
llama_model_loader: loaded meta data with 35 key-value pairs and 147 tensors from ./Llama-3.2-1B-Instruct-IQ3_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 1B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   5:                         general.size_label str              = 1B
llama_model_loader: - kv   6:                            general.l

Define Data Sources

In [3]:
import pandas as pd

file_names = [
    "study_permit_general",
    "work_permit_student_general",
    "work-study-data-llm",
    "vancouver_transit_qa_pairs",
    "permanent_residence_student_general",
    "data-with-sources"
]

all_texts = []

for file in file_names:
    path = f'./sample_data/{file}.csv'
    try:
        df = pd.read_csv(path)
        df.columns = df.columns.str.lower()

        if 'question' in df.columns and 'answer' in df.columns:
            df = df.drop_duplicates(subset=['question'])
            df['text'] = df['question'].fillna('') + ' ' + df['answer'].fillna('')
        else:
            print(f"no text columns in {file}")
            continue
        all_texts.extend(df['text'].tolist())
    except Exception as e:
        print(f"Error loading {file}: {e}")


Set Embedding Model, and Chroma Client to Interact w Vector Database and Create Collections

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb
import uuid
import pandas as pd
#from fuzzywuzzy import fuzz

# pt model for generating embeddings used pretty often
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-MiniLM-L6-v2"
)

# persistent client to interact w chroma vector store
client = chromadb.PersistentClient(path="./chroma_db")

# create collections for each data (for testing rn)
collection = client.get_or_create_collection(name="combined_docs")

# seems like better results if we remove duplicates and very similar data
data = pd.DataFrame({"text": all_texts})
data = data.drop_duplicates()
all_texts = data["text"].tolist()

print(f"successfully added {len(all_texts)} documents to Chroma DB.")

/var/folders/xp/vvb2w80x6791bg_m5v5vn0g40000gn/T/ipykernel_99366/3032524808.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


successfully added 442 documents to Chroma DB.


Function to add data to collection by embedding them

In [5]:
def add_data_to_collection_batch(collection, texts, batch_size=3):
    for idx in range(0, len(texts), batch_size):
        try:
            batch_texts = texts[idx: idx + batch_size]

            embeddings = embedding_model.embed_documents(batch_texts)

            batch_ids = [str(uuid.uuid4()) for _ in batch_texts]

            collection.add(
                ids=batch_ids,
                embeddings=embeddings,
                documents=batch_texts
            )
            print(f"successfully added {len(batch_texts)} documents (Batch {idx}-{idx + batch_size - 1})")
        except Exception as e:
            print(f"Error processing batch starting at index {idx}: {e}")

In [6]:
add_data_to_collection_batch(collection, all_texts)
print(f"successfully added {len(all_texts)} documents to the Chroma collection.")

successfully added 3 documents (Batch 0-2)
successfully added 3 documents (Batch 3-5)
successfully added 3 documents (Batch 6-8)
successfully added 3 documents (Batch 9-11)
successfully added 3 documents (Batch 12-14)
successfully added 3 documents (Batch 15-17)
successfully added 3 documents (Batch 18-20)
successfully added 3 documents (Batch 21-23)
successfully added 3 documents (Batch 24-26)
successfully added 3 documents (Batch 27-29)
successfully added 3 documents (Batch 30-32)
successfully added 3 documents (Batch 33-35)
successfully added 3 documents (Batch 36-38)
successfully added 3 documents (Batch 39-41)
successfully added 3 documents (Batch 42-44)
successfully added 3 documents (Batch 45-47)
successfully added 3 documents (Batch 48-50)
successfully added 3 documents (Batch 51-53)
successfully added 3 documents (Batch 54-56)
successfully added 3 documents (Batch 57-59)
successfully added 3 documents (Batch 60-62)
successfully added 3 documents (Batch 63-65)
successfully adde

Function to now match for releveant document

In [7]:
def get_relevant_documents(query, n_results=3):
    try:
        query_embeddings = embedding_model.embed_documents([query])[0]

        results = collection.query(query_embeddings=[query_embeddings], n_results=n_results)
        print(f"Query Results: {results}")

        return results['documents'][0] if results['documents'] else []
    except Exception as e:
        print(f"Error querying: {e}")
        return []

Generate Answer

In [8]:
def generate_answer(query):
    response_before_rag = model(query, max_tokens=200, temperature=0.1)["choices"][0]["text"]

    relevant_documents = get_relevant_documents(query)
    if not relevant_documents:
        return {
            "Before RAG Response": response_before_rag,
            "After RAG Response": "Sorry, no relevant documents found."
        }

    relevant_texts = "\n\n".join(relevant_documents)
    rag_prompt = f"""
    You are a helpful assistant for international students. Here are relevant documents:

    {relevant_texts}

    Please respond to the following question based on the documents above. Be conversational but concise:

    Question: {query}

    Answer:
    """

    response_after_rag = model(rag_prompt, max_tokens=300, temperature=0.1)["choices"][0]["text"]

    return {
        "Before RAG Response": response_before_rag,
        "After RAG Response": response_after_rag
    }


Example Usage

In [9]:
test_queries = [
    "How do I apply for a study permit in Canada?",
    "Can I work while studying on a student visa?",
    "What happens if my study permit expires before I finish my program?",
    "Do I need a new study permit if I change schools?",
    "How long does it take to process a Canadian study permit?",
    "Am I allowed to work off-campus as an international student?",
    "How many hours can I work while studying in Canada?",
    "What documents do I need to apply for a co-op work permit?",
    "Can I work in Canada after I graduate?",
    "What is a Post-Graduation Work Permit (PGWP) and how do I apply?",
    "How do I apply for MSP (Medical Services Plan) in British Columbia?",
    "Is MSP mandatory for international students?",
    "What healthcare services are covered under MSP?",
    "What should I do if I get sick and don’t have insurance yet?",
    "Can I use private health insurance instead of MSP?",
    "What are my options for student housing in Vancouver?",
    "How much does rent typically cost for international students?",
    "What should I check before signing a lease in Canada?",
    "Are there any student discounts for accommodation?",
    "How can I find a roommate in Canada?",
    "How do I open a bank account as an international student?",
    "What documents do I need to get a student bank account?",
    "Can I get a credit card as an international student?",
    "How do I send money to my home country from Canada?",
    "What scholarships are available for international students?",
    "How does the Compass Card work for transit in Vancouver?",
    "Am I eligible for a U-Pass as an international student?",
    "What is the best way to get around Vancouver on a budget?",
    "Where can I find the bus and SkyTrain schedules?",
    "Are there student discounts for public transportation?",
    "Can I apply for permanent residence after graduating?",
    "What is the Canadian Experience Class (CEC) immigration program?",
    "How can I improve my chances of getting permanent residence?",
    "What are the eligibility requirements for Express Entry?",
    "Does having a Canadian degree help with PR applications?"
]

for idx, user_query in enumerate(test_queries, start=1):
    responses = generate_answer(user_query)

    print("\n" + "="*50)
    print(f"Test Query {idx}: {user_query}")
    print("="*50)
    print("Response Before RAG:\n", responses["Before RAG Response"])
    print("\nResponse After RAG:\n", responses["After RAG Response"])
    print("="*50 + "\n\n")

#add_all_texts_to_collection()

# big speedup


llama_perf_context_print:        load time =     880.07 ms
llama_perf_context_print: prompt eval time =     879.26 ms /    12 tokens (   73.27 ms per token,    13.65 tokens per second)
llama_perf_context_print:        eval time =    4181.27 ms /   199 runs   (   21.01 ms per token,    47.59 tokens per second)
llama_perf_context_print:       total time =    5107.56 ms /   211 tokens
Llama.generate: 1 prefix-match hit, remaining 659 prompt tokens to eval


Query Results: {'ids': [['053f6297-87e3-452c-b615-4cac8daeb372', 'e6421347-dc34-492b-9dbd-6ea09fad492d', 'f138a4b0-feef-4818-a5a3-3a2d0fa6d90b']], 'embeddings': None, 'documents': [['How do I apply for a study permit?("https://ircc.canada.ca/english/helpcentre/answer.asp?qnum=479&top=15") You should apply for a study permit\xa0before coming to Canada. Make sure\xa0you have a letter of acceptance\xa0from a\xa0designated learning institution("https://www.canada.ca/en/services/immigration-citizenship/helpcentre/glossary.html#designated_learning_institution")\xa0in Canada\xa0before you apply.\nYou should\xa0apply online("https://www.canada.ca/en/immigration-refugees-citizenship/services/study-canada/study-permit/apply.html")\xa0for a study permit, whether you’re outside of Canada or already in Canada. If you\xa0can’t\xa0apply online because of a disability or because there’s a\xa0\xa0problem with the online application("https://ircc.canada.ca/english/helpcentre/answer.asp?qnum=479&top=15#i

llama_perf_context_print:        load time =     880.07 ms
llama_perf_context_print: prompt eval time =    1870.77 ms /   659 tokens (    2.84 ms per token,   352.26 tokens per second)
llama_perf_context_print:        eval time =    6209.14 ms /   299 runs   (   20.77 ms per token,    48.15 tokens per second)
llama_perf_context_print:       total time =    8157.83 ms /   958 tokens
Llama.generate: 1 prefix-match hit, remaining 10 prompt tokens to eval



Test Query 1: How do I apply for a study permit in Canada?
Response Before RAG:
  The process typically involves the following steps:
1. Check if you need a study permit: Determine if you need a study permit to study in Canada. You can check the Immigration, Refugees and Citizenship Canada (IRCC) website to see if you qualify.
2. Gather required documents: Gather the necessary documents, including:
	* A valid passport
	* A completed application form (IRCST-1)
	* Proof of language proficiency (e.g., English or French language test results)
	* Proof of sufficient funds to support yourself during your stay in Canada
	* Proof of health insurance
	* Police certificate (if required)
3. Submit your application: Submit your application online through the IRCC website or by mail. You can also submit your application by mail, but it's recommended to use the online option.
4. Pay the application fee: The application fee for a study permit is currently $150 CAD.
5. Wait for your application to be

llama_perf_context_print:        load time =     880.07 ms
llama_perf_context_print: prompt eval time =     163.84 ms /    10 tokens (   16.38 ms per token,    61.03 tokens per second)
llama_perf_context_print:        eval time =    3894.43 ms /   199 runs   (   19.57 ms per token,    51.10 tokens per second)
llama_perf_context_print:       total time =    4102.19 ms /   209 tokens


...

llama_perf_context_print:        load time =     880.07 ms
llama_perf_context_print: prompt eval time =     654.85 ms /   201 tokens (    3.26 ms per token,   306.94 tokens per second)
llama_perf_context_print:        eval time =    2544.44 ms /   124 runs   (   20.52 ms per token,    48.73 tokens per second)
llama_perf_context_print:       total time =    3224.46 ms /   325 tokens
Llama.generate: 1 prefix-match hit, remaining 11 prompt tokens to eval



Test Query 22: What documents do I need to get a student bank account?
Response Before RAG:
 ?
To get a student bank account, you'll need to provide some basic information and identification. Here are the documents you'll typically need:

1. **Proof of Identity**: A valid government-issued ID, such as a driver's license, state ID, or passport.
2. **Proof of Residency**: A utility bill, lease agreement, or other document that shows your current address.
3. **Social Security Number or Individual Taxpayer Identification Number (ITIN)**: You'll need to provide one of these numbers to verify your identity and ensure you're eligible to open a bank account.
4. **Birth Certificate**: A copy of your birth certificate, which includes your full name, date of birth, and address.
5. **Parent or Guardian's Signature**: If you're a minor, you may need a parent or guardian's signature to sign the account application.

**Additional Requirements**:

* Some banks may require additional documentation, su

llama_perf_context_print:        load time =     880.07 ms
llama_perf_context_print: prompt eval time =     162.70 ms /    11 tokens (   14.79 ms per token,    67.61 tokens per second)
llama_perf_context_print:        eval time =    4231.75 ms /   199 runs   (   21.27 ms per token,    47.03 tokens per second)
llama_perf_context_print:       total time =    4439.23 ms /   210 tokens
Llama.generate: 1 prefix-match hit, remaining 214 prompt tokens to eval


Query Results: {'ids': [['35cbd51a-176c-4097-a63e-f43d245af59a', 'cb63e232-d0f2-4b84-8c27-92aac0a7b769', '02e67a56-332d-4d9f-bca5-7b4e6d407858']], 'embeddings': None, 'documents': [['Where can international students work on-campus? International students can work at any building on their school campus, or at associated locations like libraries, hospitals, or research facilities, if they are working as a teaching or research assistant related to a research grant.', 'Where can international students work on-campus? International students can work at any building on their school campus, or at associated locations like libraries, hospitals, or research facilities, if they are working as a teaching or research assistant related to a research grant.', 'How do international students apply for the health fee? International students enrolled in MSP with a study permit valid for six months or longer will be invoiced for the health fee. Information on how to apply for MSP can be found online. For

KeyboardInterrupt: 